In [1]:
import pandas as pd
import numpy as np
import csv
import math
from sklearn.linear_model import PassiveAggressiveRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime, timedelta
from pykrige.ok import OrdinaryKriging
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# import xgboost as xgb

import constants

In [2]:
def load_static_data():
    static_sensor_data=[]
    for sensor in constants.SENSOR_IDS:
        list_ = []
        for date in constants.SELECT_DATES:
            filename = "/Users/ryanegan/Documents/diss/RawData/static/{}/{}.csv".format(date, sensor)
            df = pd.read_csv(filename,index_col=None)
            df['lat'] = constants.STATIC_COORDS[sensor][0]
            df['long'] = constants.STATIC_COORDS[sensor][1]
            # Calibration
            df['PM1'] = df['PM1'].values / constants.CALIBRATION_FACTORS[sensor][0]
            df['PM2.5'] = df['PM2.5'].values / constants.CALIBRATION_FACTORS[sensor][1]
            df['PM10'] = df['PM10'].values / constants.CALIBRATION_FACTORS[sensor][2]
            df['temperature'] = df['temperature'].values / constants.CALIBRATION_FACTORS[sensor][3]
            df['humidity'] = df['humidity'].values / constants.CALIBRATION_FACTORS[sensor][4]
            # -----------
            list_.append(df)
        static_sensor_data.append(pd.concat(list_, axis = 0, ignore_index = False))
    all_static_data = pd.concat(static_sensor_data, axis = 0, ignore_index = False)
    # Select columns
    all_static_data = all_static_data[constants.COLUMNS]
    all_static_data = all_static_data.dropna()
    return all_static_data

def load_mobile_sensors():
    list_ = []
    for sensor in constants.MOBILE_SENSORS:
        for date in constants.SELECT_DATES:
            filename = "/Users/ryanegan/Documents/diss/RawData/personal/{}/{}_{}.csv".format(date, sensor, date)
            df = pd.read_csv(filename,index_col=None)
            list_.append(df)
        mobile_sensor_data = (pd.concat(list_, axis = 0, ignore_index = False))
        mobile_sensor_data['lat'] = mobile_sensor_data['latitude']
        mobile_sensor_data['long'] = mobile_sensor_data['longitude']
        # Calibration
        df['PM1'] = df['PM1'].values / constants.CALIBRATION_FACTORS[sensor][0]
        df['PM2.5'] = df['PM2.5'].values / constants.CALIBRATION_FACTORS[sensor][1]
        df['PM10'] = df['PM10'].values / constants.CALIBRATION_FACTORS[sensor][2]
        df['temperature'] = df['temperature'].values / constants.CALIBRATION_FACTORS[sensor][3]
        df['humidity'] = df['humidity'].values / constants.CALIBRATION_FACTORS[sensor][4]
        #Select columns
        mobile_sensor_data = mobile_sensor_data[constants.COLUMNS]
    return mobile_sensor_data

#Transform to grid coordinates
def grid_lat_coord(lat):
    if lat < 55.93814 or lat > 55.94686:
        return -1
    i = 0
    for lat_check in np.linspace(55.93814,55.94686,constants.GRID_SIZE+1)[1:]:
        if lat < lat_check:
            return constants.GRID_SIZE-1-i
        i+=1

def grid_long_coord(long):
    if long < -3.19665 or long > -3.18123:
        return -1
    i = 0
    for long_check in np.linspace(-3.19665,-3.18123,constants.GRID_SIZE+1)[1:]:
        if long < long_check:
            return i
        i+=1

def transform_to_grid_coordinates(data):
    data['lat_grid'] = data['lat'].apply(grid_lat_coord)
    data['long_grid'] = data['long'].apply(grid_long_coord)
    data = data[data['lat_grid'] >= 0]
    data = data[data['long_grid'] >= 0]
    return data

def setIds(data):
    startTime = constants.START_TIME
    endTime = (datetime.strptime(startTime, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=5)).strftime("%Y-%m-%d %H:%M:%S")
    i = 0
    j = 0
    data['timestep'] = 0
    data['timestepContinuous'] = 0
    
    while startTime < constants.END_TIME:
        
        data['timestep'] = np.where((data['Timestamp'] >= startTime) & (data['Timestamp'] < endTime), i, data['timestep'])
        i+=1
        
        if (data[(data['Timestamp'] >= startTime) & (data['Timestamp'] < endTime)].shape[0] != 0):
            data['timestepContinuous'] = np.where((data['Timestamp'] >= startTime) & (data['Timestamp'] < endTime), j, data['timestepContinuous'])
            j+=1
        startTime = endTime
        endTime = (datetime.strptime(endTime, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=constants.WINDOW)).strftime("%Y-%m-%d %H:%M:%S")

def add_land_type(data):
    data['land_type'] = np.vectorize(get_land_type)(data['lat_grid'], data['long_grid'])
    
    one_hot = pd.get_dummies(data['land_type'])
    data = data.join(one_hot)
    data.rename(columns={1.0: 'commercial', 2.0: 'residential', 3.0: 'green'}, inplace=True)
    data.drop(['land_type'],axis=1, inplace=True)
    
    return data

def get_land_type(x, y):
    return landuse[int(x)][int(y)]

def add_road_type(data):
    osm_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/osm/"
    road_file = osm_dir + "roadtype_grid" + str(constants.GRID_SIZE) + ".csv"

    pd_df2=pd.read_csv(road_file, sep=',',header=None, skiprows=1)
    roadtype = pd_df2.values
    
    data['road_type'] = np.vectorize(get_road_type)(data['lat_grid'], data['long_grid'])
    
    one_hot = pd.get_dummies(data['road_type'])
    data = data.join(one_hot)
    data.rename(columns={0.0: 'primary', 1.0: 'secondary', 2.0: 'tertiary', 3.0: 'unclassified', 4.0: 'residential/service', 5.0: 'pedestrian/cycle/noroad'}, inplace=True)
    data.drop(['road_type'],axis=1, inplace=True)
    
    return data

def get_road_type(x, y):
    return roadtype[int(x)][int(y)]

def add_labels(data):
    data['next_pm_2.5'] = np.vectorize(get_next_pm)(data['lat_grid'], data['long_grid'], data['timestep'])
    return data

def get_next_pm(lat, long, timestep):
    for i in range(constants.GRID_SIZE):
        next_data = all_data[(np.abs(all_data['lat_grid'] - lat) <= i) & (np.abs(all_data['long_grid'] - long) <= i) & (all_data['timestep'] == timestep + 1)]
        if(not next_data.empty):
            next_pm = next_data['PM2.5'].mean()
            return next_pm

def add_previous(data):
    data['prev_pm_2.5'] = np.vectorize(get_previous_pm)(data['lat_grid'], data['long_grid'], data['timestep'])
    return data

def get_previous_pm(lat, long, timestep):
    for i in range(constants.GRID_SIZE):
        prev_data = all_data[(np.abs(all_data['lat_grid'] - lat) <= i) & (np.abs(all_data['long_grid'] - long) <= i) & (all_data['timestep'] == timestep - 1)]
        if(not prev_data.empty):
            prev_pm = prev_data['PM2.5'].mean()
            return prev_pm

In [3]:
staticData = load_static_data()
staticData = transform_to_grid_coordinates(staticData)
mobileData = load_mobile_sensors()
mobileData = transform_to_grid_coordinates(mobileData)
all_data = pd.concat([staticData, mobileData], ignore_index=True)
setIds(all_data)

all_data['hour'] = all_data.apply(lambda row: datetime.strptime(row.Timestamp[:19], '%Y-%m-%d %H:%M:%S').hour, axis=1)
all_data['minute'] = all_data.apply(lambda row: datetime.strptime(row.Timestamp[:19], '%Y-%m-%d %H:%M:%S').minute, axis=1)

columns= constants.UPDATED_COLUMNS
allData15 = pd.DataFrame(columns=columns)
for timestep in range(all_data['timestepContinuous'].max()):
    print(timestep)
    for x in range(constants.GRID_SIZE):
        for y in range(constants.GRID_SIZE):
            curData = all_data[(all_data['lat_grid'] == x) & (all_data['long_grid'] == y) & (all_data['timestepContinuous'] == timestep)]
            if(not curData.empty):
                curData = curData.mean(axis = 0)
                allData15.loc[len(allData15)] = curData
all_data = allData15




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
osm_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/osm/"
LU_file = osm_dir + "landuse_grid" + str(constants.GRID_SIZE) + ".csv"
road_file = osm_dir + "roadtype_grid" + str(constants.GRID_SIZE) + ".csv"

pd_df1=pd.read_csv(LU_file, sep=',',header=None, skiprows=1)
landuse = pd_df1.values
pd_df2=pd.read_csv(road_file, sep=',',header=None, skiprows=1)
roadtype = pd_df2.values


all_data = add_land_type(all_data)
all_data = add_road_type(all_data)
all_data = add_labels(all_data).dropna()
all_data = add_previous(all_data)

all_data = all_data.dropna()

with open("all_data_15_min_20_grid.txt", "wb") as fp:   #Pickling
    pickle.dump(all_data, fp)
    

In [5]:
all_data

,PM1,PM2.5,PM10,temperature,humidity,lat,long,lat_grid,long_grid,hour,...,commercial,residential,green,primary,tertiary,unclassified,residential/service,pedestrian/cycle/noroad,next_pm_2.5,prev_pm_2.5
6,0.560021,1.047775,1.250596,15.442787,81.600532,55.945433,-3.191392,3.0,6.0,0.000000,...,1,0,0,0,1,0,0,0,1.081502,1.11201
7,0.638649,1.085895,1.442452,15.610231,80.723547,55.945302,-3.188279,3.0,10.0,0.000000,...,1,0,0,0,0,0,0,1,1.095115,1.04359
8,0.673539,1.197392,1.617458,14.926126,84.561992,55.943003,-3.192147,8.0,5.0,0.000000,...,0,0,1,0,0,0,0,1,1.409194,1.22668
9,0.620221,1.092234,1.629773,15.181242,83.022128,55.943014,-3.185994,8.0,13.0,0.000000,...,1,0,0,0,0,1,0,0,1.320456,1.2062
10,0.895199,1.526833,2.664416,14.441266,84.864674,55.940953,-3.186092,13.0,13.0,0.000000,...,0,0,1,0,0,0,0,1,1.548433,1.54071
11,1.254870,3.585423,9.134979,14.800000,85.000000,55.939772,-3.191544,16.0,6.0,0.000000,...,0,0,1,1,0,0,0,0,2.869808,3.48793
12,0.594983,1.081502,1.236340,15.278152,82.272694,55.945433,-3.191392,3.0,6.0,0.000000,...,1,0,0,0,1,0,0,0,1.076748,1.04777
13,0.645609,1.095115,1.605650,15.542360,81.116362,55.945302,-3.188279,3.0,10.0,0.000000,...,1,0,0,0,0,0,0,1,1.039181,1.0859
14,0.777514,1.409194,1.736277,14.793450,85.656753,55.943003,-3.192147,8.0,5.0,0.000000,...,0,0,1,0,0,0,0,1,1.281975,1.19739
15,0.767814,1.320456,1.917637,15.112393,83.698152,55.943014,-3.185994,8.0,13.0,0.000000,...,1,0,0,0,0,1,0,0,0.991517,1.09223
